In [1]:
import pandas as pd
import toml
from pathlib import Path

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.1f}'.format)


In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]
output_dir = output_path / 'RTP/person'

# Write out CSVs with a numeric value prefix to help organize outputs 
incr = 1

In [3]:
# trip data
trip = pd.read_csv(output_path / 'agg\dash\person_trips.csv')
# person data
person = pd.read_csv(output_path / 'agg\dash\person_geog.csv')
# vmt data
vmt = pd.read_csv(output_path / 'agg/dash/person_vmt.csv')

# list of equity geographies
equity_geogs = summary_config['equity_geogs']
not_equity_geogs = ["NOT in " + item for item in equity_geogs]

In [4]:
# TRIPS
df_trip = trip.copy()
# add home RGC
df_trip['is_rgc'] = 'Not in RGC'
df_trip.loc[df_trip['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'
# add trip type
df_trip.loc[df_trip['dpurp'] != 'Work', 'trip_type'] = 'Non-Work'
df_trip.loc[df_trip['dpurp'] == 'Work', 'trip_type'] = 'Work'

# PERSONS
df_person = person.copy()
# add home RGC
df_person['is_rgc'] = 'Not in RGC'
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'

# VMT
df_vmt = vmt.copy()
# add home RGC
df_vmt['is_rgc'] = 'Not in RGC'
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'

# Select only walk and bike trips
df_vmt_bp = df_vmt[df_vmt['mode'].isin(['Walk','Bike'])].copy()
# Select only drivers (dorp = 1) and auto trips
df_vmt = df_vmt[df_vmt['mode'].isin(['SOV','HOV2','HOV3+']) & (df_vmt['dorp'] == 1)].copy()

# not in equity geography
df_person[not_equity_geogs] = 1 - df_person[equity_geogs]
df_trip[not_equity_geogs] = 1 - df_trip[equity_geogs]
df_vmt[not_equity_geogs] = 1 - df_vmt[equity_geogs]
df_vmt_bp[not_equity_geogs] = 1 - df_vmt_bp[equity_geogs]

# total population by equity geography
equity_geogs_population = df_person[equity_geogs].apply(lambda x: x * df_person['psexpfac']).sum().reset_index()
equity_geogs_population.columns = ['Equity Group', 'psexpfac']

# total population by "NOT in" equity geography
not_equity_geogs_population = df_person[not_equity_geogs].apply(lambda x: x * df_person['psexpfac']).sum().reset_index()
not_equity_geogs_population.columns = ['Equity Group', 'psexpfac']

## Trips per Day by Resident

In [5]:
def trips_per_day(geog): 
    """
    Calculate trips per day by geography
    """

    # total population by geography
    df1 = df_person.groupby([geog], as_index=False)['psexpfac'].sum().set_index(geog)

    # total trips by geography
    df2 = df_trip.groupby([geog], as_index=False)['trexpfac'].sum().set_index(geog)

    # total trips by trip type and geography
    df3 = df_trip.groupby([geog, 'trip_type'], as_index=False)['trexpfac'].sum().set_index(geog)
    df3 = df3.pivot(columns='trip_type', values='trexpfac')

    # Merge the dataframes
    df = df1.merge(df2, left_index=True, right_index=True)
    df = df.merge(df3, left_index=True, right_index=True)
    # regional totals
    df.loc['Region', ['Work','Non-Work','psexpfac','trexpfac']] = df[['Work','Non-Work','psexpfac','trexpfac']].sum()

    df['Total Trips per Day'] = df['trexpfac']/df['psexpfac']
    df['Work Trips per Day'] = df['Work']/df['psexpfac']
    df['Non-Work Trips per Day'] = df['Non-Work']/df['psexpfac']
    
    return df[['Work Trips per Day', 'Non-Work Trips per Day','Total Trips per Day']]

In [6]:
df = trips_per_day('hh_county')

f_name = str(incr)+'_trips_per_day_county.csv'
df.to_csv(output_dir / f_name); incr+=1

df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_county,,,
King,0.5,3.3,3.9
Kitsap,0.5,3.2,3.7
Outside Region,0.1,2.9,2.9
Pierce,0.4,3.1,3.6
Snohomish,0.5,3.2,3.7
Region,0.5,3.3,3.8


In [7]:
df = trips_per_day('is_rgc')
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
is_rgc,,,
In RGC,0.6,3.4,4.0
Not in RGC,0.5,3.2,3.7
Region,0.5,3.3,3.8


In [8]:
df = trips_per_day('hh_rgc')

f_name = str(incr)+'_trips_per_day_rgc.csv'
df.to_csv(output_dir / f_name); incr+=1
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_rgc,,,
Auburn,0.5,3.4,3.8
Bellevue,0.7,3.4,4.1
Bothell Canyon Park,0.4,3.4,3.9
Bremerton,0.5,3.4,3.9
Burien,0.5,3.3,3.8
Everett,0.6,3.3,3.9
Federal Way,0.5,3.4,3.8
Greater Downtown Kirkland,0.7,3.4,4.1
Issaquah,0.6,3.3,3.9


In [9]:
df = trips_per_day('hh_rg_proposed')

f_name = str(incr)+'_trips_per_day_rgc.csv'
df.to_csv(output_dir / f_name); incr+=1
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_rg_proposed,,,
CitiesTowns,0.5,3.2,3.7
Core,0.5,3.3,3.8
HCT,0.5,3.2,3.7
Metro,0.6,3.3,3.9
UU,0.4,3.2,3.6
Region,0.5,3.3,3.8


In [10]:
def trips_per_day_equity_geog(geog, geog_population):
    # total work trips by equity geography
    results_df_work = df_trip[geog].apply(lambda x: (df_trip['trip_type'] == 'Work') * x * df_trip['trexpfac']).sum().reset_index()
    results_df_work.columns = ['Equity Group', 'Work']

    # total non-work trips by equity geography
    results_df_non_work = df_trip[geog].apply(lambda x: (df_trip['trip_type'] == 'Non-Work') * x * df_trip['trexpfac']).sum().reset_index()
    results_df_non_work.columns = ['Equity Group', 'Non-Work']

    # Merge the dataframes
    df = geog_population.merge(results_df_work, on='Equity Group')
    df = df.merge(results_df_non_work, on='Equity Group')

    # calculate trip rates
    df['Work Trips per Day'] = df['Work'] / df['psexpfac']
    df['Non-Work Trips per Day'] = df['Non-Work'] / df['psexpfac']
    df['Total Trips per Day'] = (df['Work'] + df['Non-Work']) / df['psexpfac']

    return df[['Equity Group','Work Trips per Day','Non-Work Trips per Day','Total Trips per Day']]

trips_per_day_equity_geog(equity_geogs, equity_geogs_population)


,Equity Group,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
0,hh_disability_reg,0.5,3.2,3.7
1,hh_elderly_reg,0.5,3.2,3.7
2,hh_english_reg,0.5,3.3,3.8
3,hh_poverty_reg,0.5,3.3,3.8
4,hh_racial_reg,0.5,3.3,3.8
5,hh_youth_reg,0.5,3.2,3.7
6,hh_poverty_50,0.5,3.3,3.8
7,hh_racial_50,0.5,3.3,3.8


In [11]:
trips_per_day_equity_geog(not_equity_geogs, not_equity_geogs_population)

,Equity Group,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
0,NOT in hh_disability_reg,0.5,3.3,3.8
1,NOT in hh_elderly_reg,0.5,3.3,3.8
2,NOT in hh_english_reg,0.5,3.2,3.7
3,NOT in hh_poverty_reg,0.5,3.3,3.8
4,NOT in hh_racial_reg,0.5,3.2,3.7
5,NOT in hh_youth_reg,0.5,3.3,3.8
6,NOT in hh_poverty_50,0.5,3.3,3.8
7,NOT in hh_racial_50,0.5,3.2,3.7


## Miles Driven per Day by Resident

In [12]:
# pd.options.display.float_format = '{:0,.1f}'.format
df_vmt = pd.read_csv(output_path / 'agg/dash/person_vmt.csv')
df_person = person.copy()

df_vmt['is_rgc'] = 0
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_person['is_rgc'] = 0
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1

# Select only drivers (dorp = 1) and auto trips
df_vmt = df_vmt[df_vmt['mode'].isin(['SOV','HOV2','HOV3+']) & (df_vmt['dorp'] == 1)]

def vmt_per_person(df_vmt, df_person, geog):
    _df_vmt = df_vmt.groupby(geog).sum()[['travdist_wt']]
    _df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
    
    return df[['Average Miles per Person']]

In [13]:
def miles_per_day(geog): 
    """
    Calculate trips per day by geography
    """

    # total population by geography
    df1 = df_person.groupby([geog], as_index=False)['psexpfac'].sum().set_index(geog)

    # total miles by geography
    df2 = df_vmt.groupby([geog], as_index=False)['travdist_wt'].sum().set_index(geog)

    # Merge the dataframes
    df = df1.merge(df2, left_index=True, right_index=True)
    # regional totals
    df.loc['Region', ['psexpfac','travdist_wt']] = df[['psexpfac','travdist_wt']].sum()

    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
    
    return df[['Average Miles per Person']]

In [14]:
df = miles_per_day('hh_county')

f_name = str(incr)+'_vmt_county.csv'
df.to_csv(output_dir / f_name); incr+=1
df

,Average Miles per Person
hh_county,
King,11.1
Kitsap,12.8
Outside Region,28.5
Pierce,13.3
Snohomish,13.7
Region,12.2


In [15]:
miles_per_day('is_rgc')

,Average Miles per Person
is_rgc,
0,13.7
1,4.4
Region,12.2


In [16]:
df = miles_per_day('hh_rgc')

f_name = str(incr)+'_vmt_rgc.csv'
df.to_csv(output_dir / f_name); incr+=1
df

,Average Miles per Person
hh_rgc,
Auburn,6.4
Bellevue,2.3
Bothell Canyon Park,12.6
Bremerton,3.4
Burien,9.5
Everett,2.7
Federal Way,7.8
Greater Downtown Kirkland,8.7
Issaquah,10.0


In [17]:
df = miles_per_day('hh_rg_proposed')

f_name = str(incr)+'_vmt_reg_geog.csv'
df.to_csv(output_dir / f_name); incr+=1
df

,Average Miles per Person
hh_rg_proposed,
CitiesTowns,17.1
Core,11.7
HCT,13.1
Metro,7.3
UU,15.4
Region,11.1



Average Miles per Person (by residence inside or outside equity geography)

In [18]:

# total miles by equity geography
results_df_miles = df_vmt[equity_geogs].apply(lambda x: x * df_vmt['travdist_wt']).sum().reset_index()
results_df_miles.columns = ['Equity Group', 'travdist_wt']
# total miles not in equity geography
results_df_miles_not = df_vmt[equity_geogs].apply(lambda x: (1-x) * df_vmt['travdist_wt']).sum().reset_index()
results_df_miles_not.columns = ['Equity Group', 'travdist_wt']

# miles per person
## equity geography
df1 = equity_geogs_population.merge(results_df_miles, on='Equity Group')
df1['Inside Equity Geography'] = df1['travdist_wt'] / df1['psexpfac']
# not in equity geography
df2 = not_equity_geogs_population.copy()
df2['Equity Group'] = equity_geogs
df3 = df2.merge(results_df_miles_not, on='Equity Group')
df3['Outside Equity Geography'] = df3['travdist_wt'] / df3['psexpfac']

# final results
df = df1[['Equity Group','Inside Equity Geography']].merge(df3[['Equity Group','Outside Equity Geography']], on='Equity Group')


f_name = str(incr)+'_vmt_equity.csv'
df.to_csv(output_dir / f_name); incr+=1
df

,Equity Group,Inside Equity Geography,Outside Equity Geography
0,hh_disability_reg,11.6,12.8
1,hh_elderly_reg,13.2,11.4
2,hh_english_reg,11.2,12.9
3,hh_poverty_reg,10.5,13.5
4,hh_racial_reg,10.5,13.8
5,hh_youth_reg,14.7,10.2
6,hh_poverty_50,6.9,12.4
7,hh_racial_50,10.3,13.2


## Miles Walking and Biking per Day by Resident

In [19]:
df_vmt_bp2 = df_vmt_bp.copy()
df_vmt_bp2['mode'] = 'Walk and Bike'
# add walk and bike
df_vmt_bp2 = pd.concat([df_vmt_bp, df_vmt_bp2])

def walk_bike_per_person(geog):
    
    # miles by mode and geography
    df1 = df_vmt_bp2.groupby([geog, 'mode'], as_index=False)['travdist_wt'].sum().set_index(geog)

    # add total miles in region
    df1_region = df_vmt_bp2.groupby(['mode'])['travdist_wt'].sum().reset_index()
    df1_region[geog] = 'Region'
    df1_region = df1_region.set_index(geog)
    df1 = pd.concat([df1, df1_region])

    # population by geography
    df3 = df_person.groupby(geog, as_index=False)['psexpfac'].sum().set_index(geog)
    df3.loc['Region',:] = df3.sum(axis=0)

    # calculate average miles per person
    df = df1.merge(df3, on=geog)
    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']

    return df.pivot(columns='mode', values='Average Miles per Person')

In [20]:
df = walk_bike_per_person('hh_county')

f_name = str(incr)+'_mile_bikewalk_county.csv'
df.to_csv(output_dir / f_name); incr+=1
df

mode,Bike,Walk,Walk and Bike
hh_county,,,
King,0.2,0.7,1.0
Kitsap,0.2,0.6,0.7
Outside Region,NaN,0.1,0.1
Pierce,0.2,0.6,0.8
Region,0.2,0.6,0.9
Snohomish,0.2,0.6,0.8


In [21]:
df = walk_bike_per_person('is_rgc')
df

mode,Bike,Walk,Walk and Bike
is_rgc,,,
Region,0.2,0.6,0.9


In [22]:
df = walk_bike_per_person('hh_rgc')

f_name = str(incr)+'_mile_bikewalk_rgc.csv'
df.to_csv(output_dir / f_name); incr+=1
df

mode,Bike,Walk,Walk and Bike
hh_rgc,,,
Auburn,0.3,0.7,1.0
Bellevue,0.2,1.1,1.4
Bothell Canyon Park,0.3,0.6,1.0
Bremerton,0.2,0.9,1.1
Burien,0.3,0.7,1.0
Everett,0.2,1.0,1.2
Federal Way,0.3,0.8,1.1
Greater Downtown Kirkland,0.4,0.7,1.1
Issaquah,0.3,0.7,1.0


In [23]:
df = walk_bike_per_person('hh_rg_proposed')

f_name = str(incr)+'_mile_bikewalk_reg_geog.csv'
df.to_csv(output_dir / f_name); incr+=1
df

mode,Bike,Walk,Walk and Bike
hh_rg_proposed,,,
CitiesTowns,0.2,0.5,0.6
Core,0.2,0.6,0.9
HCT,0.2,0.5,0.8
Metro,0.2,0.9,1.1
Region,0.2,0.7,1.0
UU,0.2,0.5,0.6


In [24]:
def miles_bp_per_day_equity_geog(geog, geog_population):
    # total miles by mode and equity geography
    df1 = df_vmt_bp2[geog].apply(lambda x: (df_vmt_bp2['mode'] == 'Bike') * x * df_vmt_bp2['travdist_wt']).sum().reset_index().\
        rename(columns={0: 'Bike'})
    df2 = df_vmt_bp2[geog].apply(lambda x: (df_vmt_bp2['mode'] == 'Walk') * x * df_vmt_bp2['travdist_wt']).sum().reset_index().\
        rename(columns={0: 'Walk'})
    df3 = df_vmt_bp2[geog].apply(lambda x: (df_vmt_bp2['mode'] == 'Walk and Bike') * x * df_vmt_bp2['travdist_wt']).sum().reset_index().\
        rename(columns={0: 'Walk and Bike'})

    df = df1.merge(df2, on='index')
    df = df.merge(df3, on='index').\
        rename(columns={'index': 'Equity Group'})

    # calculate miles per person
    df[['Bike','Walk','Walk and Bike']] = df[['Bike','Walk','Walk and Bike']].apply(lambda x: x / geog_population['psexpfac'])

    return df


In [25]:
result_df_in = miles_bp_per_day_equity_geog(equity_geogs, equity_geogs_population)

result_df_in

,Equity Group,Bike,Walk,Walk and Bike
0,hh_disability_reg,0.2,0.7,0.9
1,hh_elderly_reg,0.2,0.6,0.8
2,hh_english_reg,0.2,0.7,0.9
3,hh_poverty_reg,0.2,0.7,0.9
4,hh_racial_reg,0.2,0.7,0.9
5,hh_youth_reg,0.2,0.5,0.7
6,hh_poverty_50,0.3,0.9,1.2
7,hh_racial_50,0.2,0.7,1.0


In [26]:
result_df_out = miles_bp_per_day_equity_geog(not_equity_geogs, not_equity_geogs_population)

result_df_out

,Equity Group,Bike,Walk,Walk and Bike
0,NOT in hh_disability_reg,0.2,0.6,0.8
1,NOT in hh_elderly_reg,0.2,0.7,0.9
2,NOT in hh_english_reg,0.2,0.6,0.8
3,NOT in hh_poverty_reg,0.2,0.6,0.8
4,NOT in hh_racial_reg,0.2,0.6,0.8
5,NOT in hh_youth_reg,0.2,0.7,1.0
6,NOT in hh_poverty_50,0.2,0.6,0.9
7,NOT in hh_racial_50,0.2,0.6,0.8


In [27]:
df = pd.concat([result_df_in,result_df_out])

f_name = str(incr)+'_mile_bikewalk_equity.csv'
df.to_csv(output_dir / f_name); incr+=1